In [ ]:
## exported from Abhijeet's with some changes of-course - for testing

import warnings
warnings.filterwarnings('ignore') 

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import LLM, Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from langchain_community.tools import DuckDuckGoSearchRun
from IPython.display import Markdown
from textwrap import dedent

# from langchain_community.tools.tavily_search import TavilySearchResults
# search_tool = TavilySearchResults(max_results=5)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/Users/sourav/workstuffs/env_exp/lib/python3.11/site-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_too

In [ ]:
llm = LLM(
    model="watsonx/mistralai/mistral-large",
    max_tokens=1000,
    temperature=0,
    # decoding_method="greedy"
)

#### Define Agents

In [4]:

paper_finder = Agent(
    role='Research Paper Finder',
    goal='Find relevant scientific papers related to the input paper',
    backstory=("You are an expert at finding relevant scientific papers "
    "and research works. You use web search to discover papers that are closely "
    "related to a given research topic."),
    tools=[search_tool],
    llm=llm,
    verbose=True,
    allow_delegation=True
)

analyzer = Agent(
    role='Research Paper Analyst',
    goal='Analyze scientific papers to extract key knowledge and limitations',
    backstory=("You are an expert at analyzing scientific papers to identify "
    "key findings, methodologies, and limitations."),
    tools=[scrape_tool, search_tool],
    llm=llm,
    verbose=True,
    allow_delegation=True
)

seed_idea_generator = Agent(
    role='Seed Idea Generator',
    goal='Generate initial research ideas by combining knowledge from related papers',
    backstory=("You specialize in combining knowledge from multiple "
    "sources to generate novel research ideas."),
    tools=[search_tool],
    llm=llm,
    verbose=True,
    allow_delegation=True
)

idea_iterator = Agent(
    role='Idea Refinement Specialist',
    goal='Iterate and improve upon seed ideas',
    backstory=("You excel at taking initial research ideas and "
    "developing them through iteration and improvement."),
    tools=[search_tool],
    llm=llm,
    verbose=True,
    allow_delegation=True
)

completion_specialist = Agent(
    role='Research Proposal Developer',
    goal='Transform ideas into complete research proposals',
    backstory=("You are skilled at developing comprehensive research "
    "proposals with clear methodologies and experiment plans."),
    tools=[search_tool],
    llm=llm,
    verbose=True,
    allow_delegation=True
)

#### Define Tasks

In [5]:
# Define Tasks
paper_finding_task = Task(
    description="Find related scientific papers for the given input paper '{input_paper}' using web search.",
    expected_output="A JSON list of related papers including title, authors, and summary.",
    agent=paper_finder
)

analyze_papers_task = Task(
    description="Analyze the related papers to identify key knowledge and limitations.",
    expected_output="A JSON object summarizing key knowledge and limitations from the related papers.",
    agent=analyzer
)

seed_idea_generation_task = Task(
    description="Generate initial research ideas based on the knowledge and limitations provided in the analysis.",
    expected_output="A JSON list of research ideas including idea_title, description, and rationale.",
    agent=seed_idea_generator
)

idea_iteration_task = Task(
    description="Refine the generated seed ideas by expanding and identifying key improvements.",
    expected_output="A JSON list of refined ideas including refined_idea_title, description, and improvements.",
    agent=idea_iterator
)

completion_task = Task(
    description="Develop a complete research proposal based on the refined research ideas.",
    expected_output="A JSON list of research proposals including proposal_title, methodology, and experiment_plan.",
    agent=completion_specialist
)

#### Create Crew

In [6]:

scientific_crew = Crew(
    agents=[
        paper_finder,
        analyzer,
        seed_idea_generator,
        idea_iterator,
        completion_specialist
    ],
    tasks=[
        paper_finding_task,
        analyze_papers_task,
        seed_idea_generation_task,
        idea_iteration_task,
        completion_task
    ],
    process=Process.sequential
)

In [7]:
# Define Inputs
inputs = {
    'input_paper': 'Advanced Machine Learning Techniques for Scientific Discovery'
}

# Execute the Crew
result = scientific_crew.kickoff(inputs)

# Agent: Research Paper Finder
## Task: Find related scientific papers for the given input paper 'Advanced Machine Learning Techniques for Scientific Discovery' using web search.


# Agent: Research Paper Finder
## Thought: To find relevant scientific papers related to the input paper 'Advanced Machine Learning Techniques for Scientific Discovery', I will start by searching the internet for related research papers.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Advanced Machine Learning Techniques for Scientific Discovery related research papers\"}"
## Tool Output: 
{'searchParameters': {'q': 'Advanced Machine Learning Techniques for Scientific Discovery related research papers', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Advanced machine-learning techniques in drug discovery', 'link': 'https://www.sciencedirect.com/science/article/pii/S1359644620305213', 'snippet': 'In this review, we detail the use of advanced techni

2025-02-28 12:31:53,089 - 8697497664 - llm.py-llm:348 - ERROR: LiteLLM call failed: litellm.InternalServerError: watsonxException - {"errors":[{"code":"downstream_request_failed","message":"Downstream vllm request failed: Internal Server Error","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"363cff8e8e09853df02666b40f8dadee","status_code":500}




LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.InternalServerError: watsonxException - {"errors":[{"code":"downstream_request_failed","message":"Downstream vllm request failed: Internal Server Error","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"363cff8e8e09853df02666b40f8dadee","status_code":500}


# Agent: Seed Idea Generator
## Thought: To generate initial research ideas, I need to analyze the key knowledge and limitations provided in the context. I will start by delegating the task of analyzing the research papers to the Research Paper Analyst.
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Analyze the research papers and identify key themes, gaps, and potential areas for further research.\", \"context\": \"The context includes a list of research papers on advanced machine-learning techniques in drug discovery and rel

In [8]:
print(result)

```json
[
    {
        "proposal_title": "Enhancing Data Quality for Machine Learning in Drug Discovery",
        "methodology": "This research will employ a combination of data imputation techniques, automated data quality checks, and data harmonization methods to improve the quality and availability of data used in drug discovery. Regular data assessments will be conducted to identify and address potential biases. Benchmark datasets will be developed to evaluate the performance of machine learning models. Data sharing and standardization practices will be implemented to enhance data integrity across the board. AI and machine learning techniques will be utilized to detect and correct data errors, ensuring completeness and consistency.",
        "experiment_plan": "1. Implement data imputation techniques to handle missing values. 2. Develop scripts for automated data quality checks. 3. Harmonize data formats and structures across datasets. 4. Conduct regular data assessments to identi

In [9]:
Markdown(result.raw)

```json
[
    {
        "proposal_title": "Enhancing Data Quality for Machine Learning in Drug Discovery",
        "methodology": "This research will employ a combination of data imputation techniques, automated data quality checks, and data harmonization methods to improve the quality and availability of data used in drug discovery. Regular data assessments will be conducted to identify and address potential biases. Benchmark datasets will be developed to evaluate the performance of machine learning models. Data sharing and standardization practices will be implemented to enhance data integrity across the board. AI and machine learning techniques will be utilized to detect and correct data errors, ensuring completeness and consistency.",
        "experiment_plan": "1. Implement data imputation techniques to handle missing values. 2. Develop scripts for automated data quality checks. 3. Harmonize data formats and structures across datasets. 4. Conduct regular data assessments to identify biases. 5. Create benchmark datasets for model evaluation. 6. Share best practices and standardize data handling procedures. 7. Utilize AI and machine learning for data quality improvement. 8. Ensure completeness and consistency of data."
    },
    {
        "proposal_title": "Developing Interpretable Machine Learning Models for Drug Discovery",
        "methodology": "This research will investigate model-agnostic and model-specific approaches to create interpretable machine learning models. Explainable AI (XAI) methods will be incorporated to provide clear explanations for model predictions. Interpretable deep learning techniques, such as concept whitening and attention mechanisms, will be explored. Transparency in data and model training will be ensured, and models will be developed to comply with regulatory standards. Collaboration with domain experts in drug discovery will be emphasized to develop relevant and useful models.",
        "experiment_plan": "1. Apply model-agnostic approaches like SHAP and LIME for interpretability. 2. Develop model-specific approaches for decision trees and rule-based models. 3. Incorporate XAI methods for clear explanations. 4. Explore interpretable deep learning techniques. 5. Ensure transparency in data and model training. 6. Develop models that comply with regulatory standards. 7. Collaborate with domain experts to refine models."
    },
    {
        "proposal_title": "Hybrid Approaches: Combining Traditional and Machine Learning Methods",
        "methodology": "This research will explore hybrid approaches that integrate traditional drug discovery methods with advanced machine learning techniques. Domain expertise will be incorporated to guide machine learning models. Sequential workflows will be used to preprocess data before applying machine learning techniques. Models will be continuously refined based on feedback from traditional methods. Complementary techniques will be utilized to fill gaps where traditional methods fall short. Rigorous testing and verification processes will be implemented to validate hybrid approaches.",
        "experiment_plan": "1. Integrate domain expertise to guide machine learning models. 2. Use traditional methods to preprocess data. 3. Apply machine learning techniques to the preprocessed data. 4. Continuously refine models based on feedback. 5. Utilize complementary techniques to fill gaps. 6. Validate hybrid approaches through rigorous testing and verification."
    },
    {
        "proposal_title": "Regulatory Frameworks for AI-Driven Drug Discovery",
        "methodology": "This research will investigate the development of regulatory frameworks that can accommodate AI-driven drug discovery. Standardized protocols will be developed to ensure transparency and robust validation methods for AI models. Ethical considerations related to AI in drug discovery will be addressed. Collaboration with regulatory bodies will be emphasized to develop and implement appropriate frameworks.",
        "experiment_plan": "1. Develop standardized protocols for AI-driven drug discovery. 2. Ensure transparency in AI models and decision-making processes. 3. Establish robust validation methods for AI models. 4. Address ethical concerns related to AI in drug discovery. 5. Collaborate with regulatory bodies to implement frameworks."
    },
    {
        "proposal_title": "Autonomous Drug Discovery Systems",
        "methodology": "This research will advance the development of fully autonomous systems that can handle the entire drug discovery pipeline, from target identification to clinical trials. AI and automation will be integrated to streamline the drug discovery process. Continuous learning and adaptation mechanisms will be implemented to improve system performance over time